# CPG: Central Pattern Generators

In [ ]:
# Standard library
from functools import partial
from pathlib import Path

# Third-party libraries
import matplotlib.pyplot as plt
import nevergrad as ng
import numpy as np
import torch
from rich.console import Console
from rich.progress import track
from rich.traceback import install

# Local libraries
from ariel.simulation.controllers import NaCPG
from ariel.simulation.controllers.na_cpg import create_fully_connected_adjacency

In [ ]:
# --- DATA SETUP --- #
CWD = Path.cwd()
DATA = CWD / "__data__"
DATA.mkdir(exist_ok=True)

# --- RANDOM GENERATOR SETUP --- #
SEED = 42
RNG = np.random.default_rng(SEED)

# --- TERMINAL OUTPUT SETUP --- #
install(show_locals=False)
console = Console()

## Parameter Tuning

In [ ]:
def scaled_beta(
    a: float,
    b: float,
    size: int | list[int],
    c: float = 0,
    d: float = 1,
    *,
    negative_reflect: bool = True,
):
    # a & b must be grater than 0
    lower = min(c, d)
    upper = max(c, d)

    # If negative_reflect is True, c and d must be >= 0
    if negative_reflect and (c < 0 or d < 0):
        msg = "c and d must be non-negative when negative_reflect is True"
        raise ValueError(msg)

    # Generate samples from the scaled beta distribution
    sample = lower + (upper - lower) * RNG.beta(a=a, b=b, size=size)

    # Reflect some values to negative if specified
    if negative_reflect:
        reflection_mask = RNG.choice([-1, 1], size=size)
        return sample * reflection_mask
    return sample

In [ ]:
def init_state(value: float, size: int | list[int]):
    return RNG.choice([-value, value], size=size)

In [ ]:
num_of_modules = 30
adj_dict = create_fully_connected_adjacency(num_of_modules)
na_cpg = NaCPG(adj_dict, angle_tracking=True, clipping=False)
scaled_beta_n = partial(scaled_beta, size=na_cpg.n)
init_state_n = partial(init_state, size=(na_cpg.n, 2))

In [ ]:
budget = 100
num_workers = 100
optimisers = {
    0: ng.optimizers.NgIohTuned,  # meta optimizer
    1: ng.optimizers.TwoPointsDE,  # overall
    2: ng.optimizers.PortfolioDiscreteOnePlusOne,  # hyperparameter
    3: ng.optimizers.CMA,  # control
    4: ng.optimizers.TBPSA,  # problems corrupted by noise
    5: ng.optimizers.PSO,  # robustness
    6: ng.optimizers.ScrHammersleySearchPlusMiddlePoint,  # super parallel
    7: ng.optimizers.RandomSearch,
}
optim = optimisers[1]

In [ ]:
parametrization = ng.p.Instrumentation(
    a=ng.p.Scalar(init=(RNG.uniform(low=0.2, high=4)), lower=0.1, upper=4),
    b=ng.p.Scalar(init=(RNG.uniform(low=0.2, high=4)), lower=0.1, upper=4),
    c=ng.p.Scalar(
        init=(RNG.uniform(low=0, high=1)),
        lower=0,
        upper=2,
    ),
    d=ng.p.Scalar(
        init=(RNG.uniform(low=0, high=1)),
        lower=0,
        upper=2,
    ),
)

optim_phase = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)
optim_amplitudes = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)
optim_w = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)
optim_ha = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)
optim_b = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)

In [ ]:
low = -100
high = 100
parametrization = ng.p.Instrumentation(
    coefficients=ng.p.Array(
        init=(RNG.uniform(low=low, high=high, size=4)),
        lower=low,
        upper=high,
    ),
)

optim_cf = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)


In [ ]:
low = 0.000_1
high = np.pi
parametrization = ng.p.Instrumentation(
    value=ng.p.Scalar(init=(RNG.uniform(low=low, high=high))),
)

optim_xy = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)
optim_xy_dot_old = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)


In [ ]:
low = 0.001
high = 0.1
factor = 10
parametrization = ng.p.Instrumentation(
    alpha=ng.p.Scalar(
        init=(RNG.uniform(low=low, high=high)),
        lower=(low / factor),
        upper=(high * factor),
    ),
    dt=ng.p.Scalar(
        init=(RNG.uniform(low=low, high=high)),
        lower=low,
        upper=high,
    ),
)

optim_alpha_dt = optim(
    parametrization=parametrization,
    budget=budget,
    num_workers=num_workers,
)

In [ ]:
best_mean = np.inf
best_std = np.inf
repeat = 10
control_frequency = 0.02
total_time = 30  # seconds
time_steps = int(total_time / control_frequency)
for _ in track(range(budget)):
    # Non-learning (overarching parameters)
    alpha_dt_asks = optim_alpha_dt.ask()
    alpha = alpha_dt_asks.kwargs["alpha"]
    dt = alpha_dt_asks.kwargs["dt"]
    cf_ask = optim_cf.ask()
    coefficients = cf_ask.kwargs["coefficients"]

    # Non-learning (state initialization)
    state_asks = {
        "xy": optim_xy.ask(),
        "xy_dot_old": optim_xy_dot_old.ask(),
    }

    # Learning
    parameter_groups_asks = {
        "phase": optim_phase.ask(),
        "w": optim_w.ask(),
        "amplitudes": optim_amplitudes.ask(),
        "ha": optim_ha.ask(),
        "b": optim_b.ask(),
    }

    # Test loop
    losses = []
    losses_clip = []
    losses_curvature = []
    losses_dev = []
    for _ in range(repeat):
        # Reset CPG
        na_cpg.reset_state()

        # Set alpha, dt and constraint function coefficients
        na_cpg.alpha = alpha
        na_cpg.dt = dt
        na_cpg.coefficients = coefficients

        # Set state
        xy = init_state_n(**state_asks["xy"].kwargs)
        xy_dot_old = init_state_n(**state_asks["xy_dot_old"].kwargs)
        na_cpg.set_state(
            xy=xy,
            xy_dot_old=xy_dot_old,
        )

        # Parameter groups
        parameter_groups = {
            "phase": scaled_beta_n(**parameter_groups_asks["phase"].kwargs),
            "w": scaled_beta_n(**parameter_groups_asks["w"].kwargs),
            "amplitudes": scaled_beta_n(
                **parameter_groups_asks["amplitudes"].kwargs,
            ),
            "ha": scaled_beta_n(**parameter_groups_asks["ha"].kwargs),
            "b": scaled_beta_n(**parameter_groups_asks["b"].kwargs),
        }
        # Set the new parameters in the CPG
        na_cpg.set_param_with_dict(parameter_groups)

        # Run the CPG to evaluate performance
        for _ in range(time_steps):
            na_cpg.forward()

        # Clipping error
        loss_clip = na_cpg.clamping_error
        losses_clip.append(loss_clip)

        # Flat-line error
        w = np.arange(len(na_cpg.angle_history))
        y = np.mean(na_cpg.angle_history, axis=1)

        # Linear fit
        m, b = np.polyfit(w, y, 1)

        # Deviation from linearity
        dev = np.mean(np.abs(m * w - y + b) / np.sqrt(m * m + 1))
        losses_dev.append(dev)

        # Curvature
        curvature = (
            np.mean(
                (w[2:] - 2 * w[1:-1] + w[:-2]) ** 2
                + (y[2:] - 2 * y[1:-1] + y[:-2]) ** 2,
            )
            if len(w) > 2
            else 0
        )
        # Perfectly straight lines are no good
        if round(curvature, 5) == 0:
            curvature = 1e6
        losses_curvature.append(curvature)

        # Total loss
        total_loss = np.abs(0.0002974789730298905 - curvature)
        # total_loss += np.abs(1.221188 - dev)
        # total_loss += loss_clip
        losses.append(total_loss)

    # Compute the final loss
    loss_std = np.std(losses)
    loss_mean = np.mean(losses)
    loss_clip_mean = np.mean(losses_clip)
    loss_curvature_mean = np.mean(losses_curvature)
    loss_dev_mean = np.mean(losses_dev)

    # Tell each optimizer the loss it got
    optim_phase.tell(parameter_groups_asks["phase"], loss_mean)
    optim_amplitudes.tell(parameter_groups_asks["amplitudes"], loss_mean)
    optim_w.tell(parameter_groups_asks["w"], loss_mean)
    optim_ha.tell(parameter_groups_asks["ha"], loss_mean)
    optim_b.tell(parameter_groups_asks["b"], loss_mean)
    optim_xy.tell(state_asks["xy"], loss_mean)
    optim_xy_dot_old.tell(state_asks["xy_dot_old"], loss_mean)
    optim_alpha_dt.tell(alpha_dt_asks, loss_mean)
    optim_cf.tell(cf_ask, loss_mean)

    # Log progress
    if (loss_mean < best_mean) or (
        loss_mean <= (best_mean + (best_mean * 0.05)) and loss_std < best_std
    ):
        best_mean = loss_mean
        best_std = loss_std

        console.log(f"New best loss: {best_mean:.4f}")
        console.log(f"Clip: {loss_clip_mean:.4f}")
        console.log(f"Curvature: {loss_curvature_mean:.4f}")
        console.log(f"Deviation: {loss_dev_mean:.4f}")

        hist = torch.tensor(na_cpg.angle_history)
        times = torch.arange(hist.shape[0]) * na_cpg.dt

        plt.figure()
        for j in range(hist.shape[1]):
            plt.plot(times, hist[:, j], label=f"joint {j}")
        plt.xlabel("time (s)")
        plt.ylabel("angle")
        plt.title("CPG angle histories")
        plt.grid(visible=True)
        plt.tight_layout()
        plt.savefig(DATA / "angle_histories.png")
        plt.show()

In [ ]:
# Non-learning (overarching parameters)
alpha_dt_asks = optim_alpha_dt.provide_recommendation()
alpha = alpha_dt_asks.kwargs["alpha"]
dt = alpha_dt_asks.kwargs["dt"]

cf_ask = optim_cf.provide_recommendation()
coefficients = cf_ask.kwargs["coefficients"]

# Non-learning (state initialization)
state_asks = {
    "xy": optim_xy.provide_recommendation(),
    "xy_dot_old": optim_xy_dot_old.provide_recommendation(),
}

# Learning
parameter_groups_asks = {
    "phase": optim_phase.provide_recommendation().kwargs,
    "w": optim_w.provide_recommendation().kwargs,
    "amplitudes": optim_amplitudes.provide_recommendation().kwargs,
    "ha": optim_ha.provide_recommendation().kwargs,
    "b": optim_b.provide_recommendation().kwargs,
}

# Round decimal places all parameters
round_factor = 5
alpha = round(alpha, round_factor)
dt = round(dt, round_factor)
coefficients = np.round(coefficients, round_factor)
state_asks = {
    "xy": {
        k: round(v, round_factor) for k, v in state_asks["xy"].kwargs.items()
    },
    "xy_dot_old": {
        k: round(v, round_factor)
        for k, v in state_asks["xy_dot_old"].kwargs.items()
    },
}
for key in parameter_groups_asks:
    for k, v in parameter_groups_asks[key].items():
        if isinstance(v, np.ndarray):
            parameter_groups_asks[key][k] = np.round(v, round_factor)
        else:
            parameter_groups_asks[key][k] = round(v, round_factor)

console.print("Best parameters found:")
console.print(f"alpha: {alpha:.4f}")
console.print(f"dt: {dt:.4f}")
console.print(f"coefficients: {coefficients}")
console.print(f"xy: {state_asks['xy']}")
console.print(f"xy_dot_old: {state_asks['xy_dot_old']}")
console.print(f"phase: {parameter_groups_asks['phase']}")
console.print(f"w: {parameter_groups_asks['w']}")
console.print(f"amplitudes: {parameter_groups_asks['amplitudes']}")
console.print(f"ha: {parameter_groups_asks['ha']}")
console.print(f"b: {parameter_groups_asks['b']}")

In [ ]:
# Non-learning (overarching parameters)
alpha_dt_asks = optim_alpha_dt.ask()
alpha = alpha_dt_asks.kwargs["alpha"]
dt = alpha_dt_asks.kwargs["dt"]
cf_ask = optim_cf.ask()
coefficients = cf_ask.kwargs["coefficients"]

# Non-learning (state initialization)
state_asks = {
    "xy": optim_xy.ask(),
    "xy_dot_old": optim_xy_dot_old.ask(),
}

# Learning
parameter_groups_asks = {
    "phase": optim_phase.ask(),
    "w": optim_w.ask(),
    "amplitudes": optim_amplitudes.ask(),
    "ha": optim_ha.ask(),
    "b": optim_b.ask(),
}

# Test loop
losses = []
losses_clip = []
losses_curvature = []
losses_dev = []

# Reset CPG
na_cpg.reset_state()

# Set alpha, dt and constraint function coefficients
na_cpg.alpha = alpha
na_cpg.dt = dt
na_cpg.coefficients = coefficients

# Set state
xy = init_state_n(**state_asks["xy"].kwargs)
xy_dot_old = init_state_n(**state_asks["xy_dot_old"].kwargs)
na_cpg.set_state(
    xy=xy,
    xy_dot_old=xy_dot_old,
)

# Parameter groups
parameter_groups = {
    "phase": scaled_beta_n(**parameter_groups_asks["phase"].kwargs),
    "w": scaled_beta_n(**parameter_groups_asks["w"].kwargs),
    "amplitudes": scaled_beta_n(
        **parameter_groups_asks["amplitudes"].kwargs,
    ),
    "ha": scaled_beta_n(**parameter_groups_asks["ha"].kwargs),
    "b": scaled_beta_n(**parameter_groups_asks["b"].kwargs),
}
# Set the new parameters in the CPG
na_cpg.set_param_with_dict(parameter_groups)

# Run the CPG to evaluate performance
for _ in range(time_steps):
    na_cpg.forward()

hist = torch.tensor(na_cpg.angle_history)
times = torch.arange(hist.shape[0]) * na_cpg.dt

plt.figure()
for j in range(hist.shape[1]):
    plt.plot(times, hist[:, j], label=f"joint {j}")
plt.xlabel("time (s)")
plt.ylabel("angle")
plt.title("CPG angle histories")
plt.grid(visible=True)
plt.tight_layout()
plt.savefig(DATA / "angle_histories.png")
plt.show()

console.log(na_cpg.clamping_error)
console.log(np.mean(np.mean(np.abs(na_cpg.angle_history), axis=1)))